In [7]:
import numpy as np
import pandas as pd


train_eeg1path = "train_eeg1.csv"
train_eeg2path = "train_eeg2.csv"
train_emgpath = "train_emg.csv"



label_path = "train_labels.csv"


test_eeg1path = "test_eeg1.csv"

test_eeg2path = "test_eeg2.csv"
test_emgpath = "test_emg.csv"




sample_path = "sample.csv"

df_eeg1 = pd.read_csv(train_eeg1path)
df_eeg2 = pd.read_csv(train_eeg2path)
df_emg = pd.read_csv(train_emgpath)



labeldf = pd.read_csv(label_path)

test_eeg1 = pd.read_csv(test_eeg1path)
test_eeg2 = pd.read_csv(test_eeg2path)
test_emg = pd.read_csv(test_emgpath)




labels = labeldf.values
labels = labels[:,1]


eeg1_data = df_eeg1.values
eeg2_data = df_eeg2.values
emg_data = df_emg.values

eeg1_test = test_eeg1.values
eeg2_test = test_eeg2.values
emg_test = test_emg.values




eeg1_data = eeg1_data[:,1:]
eeg2_data = eeg2_data[:,1:]
emg_data = emg_data[:,1:]





eeg1_test = eeg1_test[:,1:]
eeg2_test = eeg2_test[:,1:]
emg_test = emg_test[:,1:]

In [2]:
print(eeg1_data.shape)
print(eeg2_data.shape)
print(emg_data.shape)

(64800, 512)
(64800, 512)
(64800, 512)


In [3]:
print(eeg1_test.shape)

(43200, 512)


In [4]:
def class_counts(lables):
    unique, counts = np.unique(labels,return_counts=True)
    class_counts = dict(zip(unique,counts))
    print(class_counts)
    return class_counts

class_counts(labels)

{1: 34114, 2: 27133, 3: 3553}


{1: 34114, 2: 27133, 3: 3553}

In [5]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)

In [35]:
from biosppy.signals.eeg import _power_features
from biosppy.signals.emg import emg
from biosppy.signals import tools as st
import biosppy.utils as utils

def filter_signal(signal):
    b, a = st.get_filter(ftype='butter',
                         band='highpass',
                         order=8,
                         frequency=4,
                         sampling_rate=128)

    aux, _ = st._filter_signal(b, a, signal=signal, check_phase=True, axis=0)

    # low pass filter
    b, a = st.get_filter(ftype='butter',
                         band='lowpass',
                         order=16,
                         frequency=40,
                         sampling_rate=128)

    filtered, _ = st._filter_signal(b, a, signal=aux, check_phase=True, axis=0)
    return filtered


def power_features(signal=None,
                       sampling_rate=1000.,
                       size=0.25,
                       overlap=0.5,
                       bands=[[4, 8], [8, 10], [10, 13], [13, 25], [25, 40]],
                       custom=False):
    """Extract band power features from EEG signals.

    Computes the average signal power, with overlapping windows, in typical
    EEG frequency bands:
    * Theta: from 4 to 8 Hz,
    * Lower Alpha: from 8 to 10 Hz,
    * Higher Alpha: from 10 to 13 Hz,
    * Beta: from 13 to 25 Hz,
    * Gamma: from 25 to 40 Hz.

    Parameters
    ----------
    signal  array
        Filtered EEG signal matrix; each column is one EEG channel.
    sampling_rate : int, float, optional
        Sampling frequency (Hz).
    size : float, optional
        Window size (seconds).
    overlap : float, optional
        Window overlap (0 to 1).

    Returns
    -------
    ts : array
        Features time axis reference (seconds).
    theta : array
        Average power in the 4 to 8 Hz frequency band; each column is one EEG
        channel.
    alpha_low : array
        Average power in the 8 to 10 Hz frequency band; each column is one EEG
        channel.
    alpha_high : array
        Average power in the 10 to 13 Hz frequency band; each column is one EEG
        channel.
    beta : array
        Average power in the 13 to 25 Hz frequency band; each column is one EEG
        channel.
    gamma : array
        Average power in the 25 to 40 Hz frequency band; each column is one EEG
        channel.

    """

    # check inputs
    if signal is None:
        raise TypeError("Please specify an input signal.")

    # ensure numpy
    signal = np.array(signal)
    nch = signal.shape[1]
    
    # this line is new
    signal = filter_signal(signal)

    sampling_rate = float(sampling_rate)

    # convert sizes to samples
    size = int(size * sampling_rate)
    step = size - int(overlap * size)

    # padding
    min_pad = 1024
    pad = None
    if size < min_pad:
        pad = min_pad - size

    # frequency bands
    # bands = [[4, 8], [8, 10], [10, 13], [13, 25], [25, 40]]
    nb = len(bands)

    # windower
    fcn_kwargs = {'sampling_rate': sampling_rate, 'bands': bands, 'pad': pad}
    index, values = st.windower(signal=signal,
                                size=size,
                                step=step,
                                kernel='hann',
                                fcn=_power_features,
                                fcn_kwargs=fcn_kwargs)

    # median filter
    md_size = int(0.625 * sampling_rate / float(step))
    if md_size % 2 == 0:
        # must be odd
        md_size += 1

    for i in range(nb):
        for j in range(nch):
            values[:, i, j], _ = st.smoother(signal=values[:, i, j],
                                             kernel='median',
                                             size=md_size)

    # extract individual bands
    theta = values[:, 0, :]
    alpha_low = values[:, 1, :]
    alpha_high = values[:, 2, :]
    beta = values[:, 3, :]
    gamma = values[:, 4, :]
    
    delta = values[:,0,:]
    theta = values[:,1,:]
    alpha = values[:,2,:]
    spindle = values[:,3,:]
    beta1 = values[:,4,:]
    beta2 = values[:,5,:]
    # convert indices to seconds
    ts = index.astype('float') / sampling_rate

    # output
    args = (ts, theta, alpha_low, alpha_high, beta, gamma)
    args2 = (ts, delta, theta, alpha, spindle, beta1, beta2)
    
    names = ('ts', 'theta', 'alpha_low', 'alpha_high', 'beta', 'gamma')
    names2 = ("ts", "delta", "theta", "alpha", "spindle", "beta1", "beta2")
    if custom:
        return utils.ReturnTuple(args2, names2)
    else:
        return utils.ReturnTuple(args, names)

In [36]:

bands = [[0.39, 3.13], [3.13,8.46], [8.46,10.93],[10.93,15.63],[15.63,21.88],[21.88,37.50]]
"""
#for i in range(eeg1_data.shape[0]):
    signal = eeg1_data[i,:].reshape(1,eeg1_data.shape[1])
    vals = power_features(signal=signal,sampling_rate=128,bands = bands, custom = True)
    delta = vals["delta"]
    theta = vals["theta"]
    alpha = vals["alpha"]
    spindle = vals["spindle"]
    beta1 = vals["beta1"]
    beta2 = vals["beta2"]
    print(delta.shape)
    print(theta.shape)
    print(alpha.shape)
    print(spindle.shape)
    print(beta1.shape)
    print(beta2.shape)
"""
vals = power_features(signal=eeg1_data,sampling_rate=128,bands = bands, custom = True)

#vals1 = get_power_features(eeg1_data,sampling_rate=128)
#vals2 = get_power_features(eeg2_data)



In [ ]:
delta = v["delta"]
print(delta.shape)

In [37]:
delta = vals["delta"]
theta = vals["theta"]
alpha = vals["alpha"]
spindle = vals["spindle"]
beta1 = vals["beta1"]
bdeeta2 = vals["beta2"]
print(delta.shape)
print(theta.shape)
print(alpha.shape)
print(spindle.shape)
print(beta1.shape)
print(beta2.shape)

(4049, 512)
(4049, 512)
(4049, 512)
(4049, 512)
(4049, 512)
(4049, 512)


In [82]:
def energy(E):
    p = np.power(E,2)
    p = np.sum(p,axis=1)
    return np.sqrt(p)

def signal_stats(signal):
 
    # mean
    mean = np.mean(signal,axis=1)

    # median
    median = np.median(signal,axis=1)

    # maximum amplitude abs
    #maxAmpAbs = np.abs(signal - mean).max()

    # minimum amplitude abs
    #minAmpAbs = np.abs(signal - mean).min()

    # maximum amplitude
    #maxAmp = (signal - mean).max()

    # minimum amplitude
    #minAmp = (signal - mean).min()

    # variance
    sigma2 = np.var(signal,axis=1)

    # standard deviation
    sigma = np.std(signal,axis=1)

    # absolute deviation
    #ad = np.sum(np.abs(signal - median))

    # kurtosis
    #kurt = stats.kurtosis(signal, bias=False)

    # skweness
    #skew = stats.skew(signal, bias=False)
        
        
    return mean, median, sigma2, sigma

def eeg_features_(delta, theta, alpha, spindle, beta1, beta2):
    delta = vals["delta"]
    theta = vals["theta"]
    alpha = vals["alpha"]
    spindle = vals["spindle"]
    beta1 = vals["beta1"]
    beta2 = vals["beta2"]
    e1 = energy(delta)
    e2 = energy(theta)
    e3 = energy(alpha)
    e4 = energy(spindle)
    e5 = energy(beta1)
    e6 = energy(beta2)
    
    total_energy = e1+e2+e3+e4+e5+e6
    
    ratio1 = e3/(e1+e2)
    ratio2 = e1/(e2+e3)
    ratio3 = e2/(e3+e1)
    
    ratio4 = e1/(e5+e4)
    ratio5 = e2/(e4+e3)
    
    (theta_mean,theta_median,theta_sigma2,theta_sigma) = signal_stats(theta)
    (alphal_mean,alphal_median,alphal_sigma2,alphal_sigma) = signal_stats(delta)
    (alphah_mean,alphah_median,alphah_sigma2,alphah_sigma) = signal_stats(alpha)
    (beta_mean,beta_median,beta_sigma2,beta_sigma) = signal_stats(spindle)
    (gamma_mean,gamma_median,gamma_sigma2,gamma_sigma) = signal_stats(beta1)
    (beta2_mean,beta2_median,beta2_sigma2,beta2_sigma) = signal_stats(beta2)



    features = np.zeros((1,theta.shape[0]))
    
    features = np.hstack((e1,e2,e3,e4,e5,total_energy,ratio1,ratio2,ratio3,ratio4,ratio5,theta_mean,
                   theta_median,theta_sigma2,theta_sigma,alphal_mean,alphal_median,alphal_sigma2,
                   alphal_sigma, alphah_mean, alphah_median, alphah_sigma2, alphah_sigma,
                   beta_mean, beta_median, beta_sigma2, beta_sigma, gamma_mean, gamma_median,
                   gamma_sigma2, gamma_sigma,beta2_mean,beta2_median,beta2_sigma2,beta2_sigma))
    #features = features.reshape((features.shape[1],features.shape[0]))
    
    return features

def eeg_features(vals):
    delta = vals["delta"]
    theta = vals["theta"]
    alpha = vals["alpha"]
    spindle = vals["spindle"]
    beta1 = vals["beta1"]
    beta2 = vals["beta2"]
    f1 = eeg_features_(delta[:,0],theta[:,0],alpha[:,0],spindle[:,0],beta1[:,0],beta2[:,0])
    f2 = eeg_features_(delta[:,1],theta[:,1],alpha[:,1],spindle[:,1],beta1[:,1],beta2[:,1])
    f1 = f1.reshape(1,f1.shape[0])
    f2 = f2.reshape(1,f2.shape[0])
    f = np.concatenate((f1,f2),axis=1)
    return f
#features1 = eeg_features(theta1,alpha_low1,alpha_high1,beta1,gamma1)
#features2 = eeg_features(theta2,alpha_low2,alpha_high2,beta2,gamma2)

In [84]:
signal = np.zeros((eeg1_data.shape[1],2))
signal[:,0] = eeg1_data[0,:].reshape(eeg1_data.shape[1])
signal[:,1] = eeg2_data[0,:].reshape(eeg2_data.shape[1])
vals = power_features(signal=signal, sampling_rate=128,bands=bands, custom = True)
f = eeg_features(vals)
features = np.zeros((eeg1_data.shape[0],f.shape[1]))

for i in range(eeg1_data.shape[0]):
    if i%300==0:
        print("EVALUATING ROW {} PLEASE WAIT".format(i))
    signal = np.zeros((eeg1_data.shape[1],2))
    signal[:,0] = eeg1_data[i,:].reshape(eeg1_data.shape[1])
    signal[:,1] = eeg2_data[i,:].reshape(eeg2_data.shape[1])
    vals = power_features(signal=signal, sampling_rate=128,bands=bands, custom = True)
    features[i,:] = eeg_features(vals)
    

EVALUATING ROW 0 PLEASE WAIT
EVALUATING ROW 300 PLEASE WAIT
EVALUATING ROW 600 PLEASE WAIT
EVALUATING ROW 900 PLEASE WAIT
EVALUATING ROW 1200 PLEASE WAIT
EVALUATING ROW 1500 PLEASE WAIT
EVALUATING ROW 1800 PLEASE WAIT
EVALUATING ROW 2100 PLEASE WAIT
EVALUATING ROW 2400 PLEASE WAIT
EVALUATING ROW 2700 PLEASE WAIT
EVALUATING ROW 3000 PLEASE WAIT
EVALUATING ROW 3300 PLEASE WAIT
EVALUATING ROW 3600 PLEASE WAIT
EVALUATING ROW 3900 PLEASE WAIT
EVALUATING ROW 4200 PLEASE WAIT
EVALUATING ROW 4500 PLEASE WAIT
EVALUATING ROW 4800 PLEASE WAIT
EVALUATING ROW 5100 PLEASE WAIT
EVALUATING ROW 5400 PLEASE WAIT
EVALUATING ROW 5700 PLEASE WAIT
EVALUATING ROW 6000 PLEASE WAIT
EVALUATING ROW 6300 PLEASE WAIT
EVALUATING ROW 6600 PLEASE WAIT
EVALUATING ROW 6900 PLEASE WAIT
EVALUATING ROW 7200 PLEASE WAIT
EVALUATING ROW 7500 PLEASE WAIT
EVALUATING ROW 7800 PLEASE WAIT
EVALUATING ROW 8100 PLEASE WAIT
EVALUATING ROW 8400 PLEASE WAIT
EVALUATING ROW 8700 PLEASE WAIT
EVALUATING ROW 9000 PLEASE WAIT
EVALUATING ROW

In [88]:
signal = np.zeros((eeg1_test.shape[1],2))
signal[:,0] = eeg1_test[0,:].reshape(eeg1_test.shape[1])
signal[:,1] = eeg2_test[0,:].reshape(eeg2_test.shape[1])
vals = power_features(signal=signal, sampling_rate=128,bands=bands, custom = True)
f = eeg_features(vals)
test_f = np.zeros((eeg1_test.shape[0],f.shape[1]))

for i in range(eeg1_test.shape[0]):
    if i%300==0:
        print("EVALUATING ROW {} PLEASE WAIT".format(i))
    signal = np.zeros((eeg1_data.shape[1],2))
    signal[:,0] = eeg1_test[i,:].reshape(eeg1_test.shape[1])
    signal[:,1] = eeg2_test[i,:].reshape(eeg2_test.shape[1])
    vals = power_features(signal=signal, sampling_rate=128,bands=bands, custom = True)
    test_f[i,:] = eeg_features(vals)

EVALUATING ROW 0 PLEASE WAIT
EVALUATING ROW 300 PLEASE WAIT
EVALUATING ROW 600 PLEASE WAIT
EVALUATING ROW 900 PLEASE WAIT
EVALUATING ROW 1200 PLEASE WAIT
EVALUATING ROW 1500 PLEASE WAIT
EVALUATING ROW 1800 PLEASE WAIT
EVALUATING ROW 2100 PLEASE WAIT
EVALUATING ROW 2400 PLEASE WAIT
EVALUATING ROW 2700 PLEASE WAIT
EVALUATING ROW 3000 PLEASE WAIT
EVALUATING ROW 3300 PLEASE WAIT
EVALUATING ROW 3600 PLEASE WAIT
EVALUATING ROW 3900 PLEASE WAIT
EVALUATING ROW 4200 PLEASE WAIT
EVALUATING ROW 4500 PLEASE WAIT
EVALUATING ROW 4800 PLEASE WAIT
EVALUATING ROW 5100 PLEASE WAIT
EVALUATING ROW 5400 PLEASE WAIT
EVALUATING ROW 5700 PLEASE WAIT
EVALUATING ROW 6000 PLEASE WAIT
EVALUATING ROW 6300 PLEASE WAIT
EVALUATING ROW 6600 PLEASE WAIT
EVALUATING ROW 6900 PLEASE WAIT
EVALUATING ROW 7200 PLEASE WAIT
EVALUATING ROW 7500 PLEASE WAIT
EVALUATING ROW 7800 PLEASE WAIT
EVALUATING ROW 8100 PLEASE WAIT
EVALUATING ROW 8400 PLEASE WAIT
EVALUATING ROW 8700 PLEASE WAIT
EVALUATING ROW 9000 PLEASE WAIT
EVALUATING ROW

In [98]:
np.savetxt("extracted_features_test.csv",test_f,delimiter=",")
np.savetxt("extracted_features_train.csv",features,delimiter=",")


In [2]:
import numpy as np
import pandas as pd




In [19]:
features = np.loadtxt("extracted_features_train.csv",delimiter=",")
test_f = np.loadtxt("extracted_features_test.csv",delimiter=",")

In [20]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier((1024,1024),verbose=True,early_stopping=False,shuffle=False,max_iter=20)
#nn.fit(features,labels)
#nn.predict(test_f)

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=500)
pca.fit(features)
features = pca.transform(features)
test_f = pca.transform(test_f)

In [11]:
print(features.shape)

(64800, 500)


In [21]:
from sklearn.metrics import balanced_accuracy_score

def leave_one_out(data, labels, clf):
    x = np.split(data,3)
    y = np.split(labels,3)
    vals = []
    for i in range(3):
        
        x_val = x[i]
        y_val = labels[i*21600:(i+1)*21600]
        
    
        x_train = np.zeros((x_val.shape[0]*2,x_val.shape[1]))
        y_train = np.zeros((x_val.shape[0]*2,1))
        x_train[:21600,:] = x[(i+1)%3]
        x_train[21600:,:] = x[(i+2)%3]
        y_train[:21600] = y[(i+1)%3].reshape(x_val.shape[0],1)
        y_train[21600:] = y[(i+2)%3].reshape(x_val.shape[0],1)
        print("FITTING")
        clf.fit(x_train,y_train)
        y_pred = clf.predict(x_val)
        print(y_pred.shape)
        print(y_val.shape)
        val = balanced_accuracy_score(y_val,y_pred)
        print(val)
        vals.append(val)
    return vals

vals = leave_one_out(features,labels,nn)

FITTING


/Users/pouya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Iteration 1, loss = 0.88467009
Iteration 2, loss = 0.71847203
Iteration 3, loss = 0.81555472
Iteration 4, loss = 0.65929885
Iteration 5, loss = 0.63653220
Iteration 6, loss = 0.64362203
Iteration 7, loss = 0.67614475
Iteration 8, loss = 0.64597561
Iteration 9, loss = 0.63795999
Iteration 10, loss = 0.63712585
Iteration 11, loss = 0.61805410
Iteration 12, loss = 0.59548167
Iteration 13, loss = 0.63736696
Iteration 14, loss = 0.60413202
Iteration 15, loss = 0.61294541
Iteration 16, loss = 0.60742780
Iteration 17, loss = 0.59368451
Iteration 18, loss = 0.60739391
Iteration 19, loss = 0.65163722
Iteration 20, loss = 0.60647933


/Users/pouya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


(21600,)
(21600,)
0.5712538360619149
FITTING


/Users/pouya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:921: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/pouya/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:568: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

In [ ]:
nn.fit(features,labels)
pred = nn.predict(test_f)


In [ ]:
sample = pd.read_csv("sample.csv", delimiter=',')
sample['y'] = pred
sample.to_csv("NN_eegOnly.csv", index = False)